In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import copy
from matplotlib import cm, colors
import cv2
import warnings



# Takes a 2D numpy array as input (i.e. a numpy representation of an image)
# Different from tree since the empty ratio is different. Could implement a generic
# solution down the line
def processNextHouseImage(image): 
    objectwidth = 100
    objectheight = 100
    xspacing = 116
    yspacing = 300
    xstart = 293
    ystart = 445
    xend = 1730
    yend = 1770
    powderthickness = 80
    objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
        ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
    # Rest NYI

# Returns a list of images of each object, with the background filtered out
def processNextTreeImage(image, mask):
    objectwidth = 83
    objectheight = 122
    xspacing = 133
    yspacing = 270
    xstart = 293
    ystart = 268
    xend = 1730
    yend = 1770

    # objectinfo = pd.read_csv('Parameters.csv', names=["Object", "P", "S", "H", "Porosity", "Label"])

    objectCoordinates = [[x, x+objectwidth, y, y+objectheight] for y in reversed(range(
        ystart, yend, objectheight + yspacing)) for x in range(xstart, xend, xspacing + objectwidth)]
    # coorddf = pd.DataFrame(objectCoordinates, columns=['xstart', 'xend', 'ystart', 'yend'])
    # objectinfo = coorddf.join(objectinfo)

    objects = np.full((len(objectCoordinates), objectheight, objectwidth), np.nan)

    for index, object in objectCoordinates.iterrows():
        objects[index] = image[object.ystart:object.yend, object.xstart:object.xend]

    for object in objects:
        object[~mask] = np.nan

    return objects

# Conducts preprocessing. OldObjectLayers is a list of objects, with each object being a 3d numpy array

class preprocessor:
    def __init__(self, type, noOfObjects, x, y, z):
        self.type = type
        self.accumulatedLayers = [np.empty((z, x, y))]  * noOfObjects
        self.currentLayer = -1

    def preprocess(self, newObjectLayers):
        self.currentLayer += 1
        layerIndex = self.currentLayer
        returnValue = []
        for index, object in enumerate(newObjectLayers):
            if type == 'scatter' or type == 'spatstat':
                object = np.copy(object)
                xs = np.array(object, copy=True, dtype=np.float32)
                self.accumulatedLayers[index][layerIndex] = xs
                returnValue.append(xs)
            elif type == 'moran':
                # Todo: consider revising and not doing on a layer-by-layer basis?
                xs = np.array(object, copy=True, dtype=np.float32)
                avg = np.nanmean(xs)
                stddev = np.nanstd(xs)
                xs = (xs - avg) / stddev
                self.accumulatedLayers[index][layerIndex] = xs
                returnValue.append(xs)
        return returnValue

class outlierCalculator:
    def __init__(self, type, neighbourhoodDistance, windowSize, noOfObjects, z, x, y):
        self.type = type
        self.nbhd = neighbourhoodDistance
        self.ws = windowSize
        self.accumulatedLayers = [np.empty((z, x, y))]  * noOfObjects
        self.currentLayer = -1


    def calculate(self, newObjectLayers):
        self.currentLayer += 1
        outlierValues = []
        for index, objectLayer in newObjectLayers:
            object = np.copy(object)
            # Step 1: calculate neighbourhood
            neighbourkernel = np.ones((self.nbhd, self.nbhd)) / self.nbhd**2
            flatNeighbourhood = np.array(cv2.filter2D(src=objectLayer, ddepth=-1, kernel=neighbourkernel))
            self.accumulatedLayers[index][self.currentLayer] = flatNeighbourhood

            if(self.currentLayer < self.ws):
                print("no result yet")
                return outlierValues

            neighbourhoodValues = np.array(np.sum(self.accumulatedLayers[self.currentLayer-self.ws:self.currentLayer], axis=0)/self.ws)
            # Här har vi tre saker att hålla koll på:
            # - Nästa lager
            # - Tidigare lager som del av beräkningen
            # - Tidigare lagers neighbourhoodvalues (eller, varför?)
            # Vi vill returnera fler neighbourhoodvalues baserat på lagret som kom in... skit i tidigare, de poolas längre ned.

            # Step 2: calculate outlier
            ys = neighbourhoodValues
            xs = object
            filter = np.logical_and(np.isfinite(xs), np.isfinite(ys))

            # plt.imshow(xs[0])
            # plt.figure()
            # plt.imshow(xs[0])
            # plt.figure()
            # if(index == 58):
            #     plt.imshow(xs[0])
            #     plt.figure()
            #     plt.imshow(ys[0])
            #     plt.figure()
            #     plt.imshow(filter[0])
            #     plt.figure()
            #     print(len(np.unique(filter)))
            numberOfFilterValues = len(np.unique(filter))
            assert numberOfFilterValues == 2, f"Expected filter to have two values, got: {numberOfFilterValues}"
            # print("filterlength is: ", numberOfFilterValues)
            # print("index is:", index)
            if type == 'spatstat':
                outliers = xs - ys
                avg = np.mean(outliers[filter])
                std = np.std(outliers[filter])
                outliers = (outliers - avg) / std
                outlierValues.append(outliers)
            else:
                with warnings.catch_warnings():
                    line = np.polyfit(xs[filter].flatten(), ys[filter].flatten(), 1)
                    p = np.poly1d(line)
                    outlierValues.append(p(xs) - ys)
                assert(xs.shape == p(ys).shape)
            assert(len(np.unique(outlierValues[index])) > 1)
            assert(len(np.unique(np.isfinite(outlierValues[index]))) == 2)
        return outlierValues

class encoder:
    def __init__(self, noOfBins, noOfObjects, minval=0, maxval=0):
        self.min = minval
        self.max = maxval
        self.buckets = noOfBins
        self.X = np.full((noOfObjects, noOfBins), np.nan)
        self.data = np.full((noOfObjects, 0), 0)
        self.noOfLayers = 0
        # self.raw = np.array()
    
    def lockAndReset(self):
        self.locked = True
        self.X = np.full((self.noOfObjects, self.noOfBins), np.nan)
    
    def encode(self, outlierobjects):
        self.noOfLayers += 1
        numberOfObjects = len(outlierobjects)
        # If not locked, store all values and recalculate histogram
        # If locked, store only histogramstuff
        # What do we emit here? I'd say we emit the complete histogram
        newNumbers = np.concatenate([oo.flatten() for oo in outlierobjects])
        filter = np.isfinite(self.raw)
        if not self.locked:
            newmin = np.nanmin(newNumbers)
            newmax = np.nanmax(newNumbers)
            self.min = min(newmin, self.min)
            self.max = max(newmax, self.max)
            xs = outlierobjects
            filter = np.isfinite(xs)
            self.data = np.concatenate([self.data, outlierobjects[filter]])
            # "Lägg till en rad på vår numpy array..."
            for index in range(0, numberOfObjects):
                hist, edges = np.histogram(self.data[index], bins=self.buckets, range=(self.min, self.max), density=True)
                self.X[index] = np.array(hist)
        else:
            for index in range(0, numberOfObjects):
                xs = outlierobjects[index]
                filter = np.isfinite(xs)
                hist, edges = np.histogram(xs[filter], bins=self.buckets, range=(self.min, self.max), density=True)
                histnorm = hist / self.noOfLayers
                self.X[index] = self.X[index] * ((self.noOfLayers-1) / self.noOfLayers) + histnorm/self.noOfLayers
                # Hur blir matten? Behöver veta vilket lager, alternativt normalisera.

        
        return self.X

In [ ]:
# High level picture
# We can batch prepare

trainingdata = gettrainingdata()
mask = createMask(trainingdata)


on(message):
    nextImg = processNextTreeImage(message, mask)
    
